## Описание проекта

Вы работаете в интернет-магазине «Стримчик», который продаёт по всему миру компьютерные игры. Из открытых источников доступны исторические данные о продажах игр, оценки пользователей и экспертов, жанры и платформы (например, Xbox или PlayStation). Вам нужно выявить определяющие успешность игры закономерности. Это позволит сделать ставку на потенциально популярный продукт и спланировать рекламные кампании.

Перед вами данные до 2016 года. Представим, что сейчас декабрь 2016 г., и вы планируете кампанию на 2017-й. Нужно отработать принцип работы с данными. Неважно, прогнозируете ли вы продажи на 2017 год по данным 2016-го или же 2027-й — по данным 2026 года.

В наборе данных попадается аббревиатура ESRB (Entertainment Software Rating Board) — это ассоциация, определяющая возрастной рейтинг компьютерных игр. ESRB оценивает игровой контент и присваивает ему подходящую возрастную категорию, например, «Для взрослых», «Для детей младшего возраста» или «Для подростков».

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats as st
pd.options.mode.chained_assignment = None 

In [2]:
data = pd.read_csv('C:\\Users\\006\\Desktop\\Лера\\учеба Яндекс\\Проекты\\Сборный Проект №1\\games.csv')
data

,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
16710,Samurai Warriors: Sanada Maru,PS3,2016.0,Action,0.00,0.00,0.01,0.00,NaN,NaN,NaN
16711,LMA Manager 2007,X360,2006.0,Sports,0.00,0.01,0.00,0.00,NaN,NaN,NaN
16712,Haitaka no Psychedelica,PSV,2016.0,Adventure,0.00,0.00,0.01,0.00,NaN,NaN,NaN
16713,Spirits & Spells,GBA,2003.0,Platform,0.01,0.00,0.00,0.00,NaN,NaN,NaN


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


In [4]:
data.min()

Platform             2600
Year_of_Release    1980.0
NA_sales              0.0
EU_sales              0.0
JP_sales              0.0
Other_sales           0.0
Critic_Score         13.0
dtype: object

In [5]:
data.max()

Platform             XOne
Year_of_Release    2016.0
NA_sales            41.36
EU_sales            28.96
JP_sales            10.22
Other_sales         10.57
Critic_Score         98.0
dtype: object

### Вывод

Таблицу изучила, есть пропученные значения, есть столбцы, где нужно менять тип данных.
Посмотрела в общем минимум и максимум в столбцах, на первый взгляд нет ошибочных значений, оценки соответствуют указанным критериям.

## Шаг 2. Подготовьте данные

### Замените названия столбцов (приведите к нижнему регистру)

In [6]:
data.columns = map(str.lower, data.columns)

In [7]:
display(data.columns)

Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')

### Преобразуйте данные в нужные типы. Опишите, в каких столбцах заменили тип данных и почему

In [8]:
# год переведу в целочисленный тип на следующем этапе для моего удобства

# чтобы перевести user_score в float64 сначала уберу значение tbd, заменю на nan, так как на 0 менять не логично

data.loc[data['user_score'] == 'tbd', 'user_score'] = np.nan
data['user_score'] = data['user_score'].astype('float64')

In [9]:
# проверим, как изменились типы данных

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16715 non-null  object 
 2   year_of_release  16446 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16715 non-null  float64
 5   eu_sales         16715 non-null  float64
 6   jp_sales         16715 non-null  float64
 7   other_sales      16715 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       7590 non-null   float64
 10  rating           9949 non-null   object 
dtypes: float64(7), object(4)
memory usage: 1.4+ MB


## Обработайте пропуски при необходимости:
- Объясните, почему заполнили пропуски определённым образом или почему не стали это делать;
- Опишите причины, которые могли привести к пропускам;
- Обратите внимание на аббревиатуру 'tbd' в столбцах с рейтингом. Отдельно разберите это значение и опишите, как его обработать

In [10]:
# посмотрим, в каких столбцах есть пропуски

data.isna().sum()

name                  2
platform              0
year_of_release     269
genre                 2
na_sales              0
eu_sales              0
jp_sales              0
other_sales           0
critic_score       8578
user_score         9125
rating             6766
dtype: int64

1. В столбце name удалю 2 строки с отсутствующими названиями, так как в рамках выборки это 0,01%, то есть на результаты анализа не повлияют, а анализировать данные без названия игры нецелесообразно
2. столбец year_of_release попробую заменить пропуски по названию платформы
3. critic_score, user_score, rating - пропуски заменять не буду, так как замена на 0 потом скажется на результатах, а нам это не нужно. тем более, что тут так много пропусков.

In [11]:
# удаляю 2 строки с пустым значением названия игры

data = data.dropna(subset =['name'])

In [12]:
# Посмотрим, есть ли зависимость года выпуска от платформы
# Хочу это сделать для того, чтобы понять, можно ли заменить пропуски в столбце year_of_release на медианное значение, 
# рассчитанное по типу платформы

data_platform = data.pivot_table(index= 'platform', values= 'year_of_release', aggfunc= ['min', 'max']).reset_index()
display(data_platform)

,platform,min,max
,,year_of_release,year_of_release
0,2600,1980.0,1989.0
1,3DO,1994.0,1995.0
2,3DS,2011.0,2016.0
3,DC,1998.0,2008.0
4,DS,1985.0,2013.0
5,GB,1988.0,2001.0
6,GBA,2000.0,2007.0
7,GC,2001.0,2007.0
8,GEN,1990.0,1994.0


Думаю, что вполне можно опираться на такой вариант при заполнении пропусков.

In [13]:
print('Год выпуска. Пропуски до:', data['year_of_release'].isna().sum())

for platform_name in data['platform'].unique():
    median_year = data.loc[data['platform'] == platform_name, 'year_of_release'].median()  
    data.loc[(data['year_of_release'].isna()) & (data['platform'] == platform_name), 'year_of_release'] = median_year



data['year_of_release'] = data['year_of_release'].fillna(data['year_of_release'].median())    
    
print('Расстояние до ближайшего аэропорта. Пропуски после:', data['year_of_release'].isna().sum())

Год выпуска. Пропуски до: 269
Расстояние до ближайшего аэропорта. Пропуски после: 0


In [14]:
# год передевем в целочисленный тип

data['year_of_release'] = data['year_of_release'].astype('Int64')

#### Вывод

В столбце name удалены 2 строки с отсутствующими названиями, так как в рамках выборки это 0,01%, то есть на результаты анализа не повлияют, а анализировать данные без названия игры нецелесообразно
Столбец year_of_release заменены пропуски на медианное значение по платформе
critic_score, user_score, rating - пропуски оставлены, так как замена на 0 потом скажется на результатах, а нам это не нужно. тем более, что тут так много пропусков. 

Причины, которые могли привести к пропускам: в столбцах critic_score, user_score и rating - не проводилась оценка, нет и результата. По столбцам name и year_of_release - думаю, что ошибка из-за человеческого фактора: не внесли значения.

Аббревиатура 'tbd' в столбце User_Score заменила на nan в прошлом пункте.

### Посчитайте суммарные продажи во всех регионах и запишите их в отдельный столбец

In [15]:
data['total_sales'] = data['na_sales'] + data['eu_sales'] + data['jp_sales'] + data['other_sales']

In [16]:
data

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,total_sales
0,Wii Sports,Wii,2006,Sports,41.36,28.96,3.77,8.45,76.0,8.0,E,82.54
1,Super Mario Bros.,NES,1985,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN,40.24
2,Mario Kart Wii,Wii,2008,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E,35.52
3,Wii Sports Resort,Wii,2009,Sports,15.61,10.93,3.28,2.95,80.0,8.0,E,32.77
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN,31.38
...,...,...,...,...,...,...,...,...,...,...,...,...
16710,Samurai Warriors: Sanada Maru,PS3,2016,Action,0.00,0.00,0.01,0.00,NaN,NaN,NaN,0.01
16711,LMA Manager 2007,X360,2006,Sports,0.00,0.01,0.00,0.00,NaN,NaN,NaN,0.01
16712,Haitaka no Psychedelica,PSV,2016,Adventure,0.00,0.00,0.01,0.00,NaN,NaN,NaN,0.01
16713,Spirits & Spells,GBA,2003,Platform,0.01,0.00,0.00,0.00,NaN,NaN,NaN,0.01


## Шаг 3. Проведите исследовательский анализ данных

Посмотрите, сколько игр выпускалось в разные годы.

In [17]:
data.pivot_table(index= 'year_of_release', values= 'name', aggfunc= 'count').reset_index()

,year_of_release,name
0,1980,9
1,1981,46
2,1982,53
3,1983,17
4,1984,14
5,1985,14
6,1986,21
7,1987,16
8,1988,15
9,1989,17


Посмотрите, как менялись продажи по платформам. Выберите платформы с наибольшими суммарными продажами и постройте распределение по годам. За какой характерный срок появляются новые и исчезают старые платформы?

In [18]:
# Выберем платформы с наибольшими суммарными продажами

# достаем индекс
top_6 = data.pivot_table(index= 'platform', values= 'total_sales', aggfunc= 'sum').sort_values(by='total_sales', ascending=False).head(6).index.values

# Взяла 6, так как предварительно проанализировала и после 6 строки был резкий спад суммарных продаж

top_6

array(['PS2', 'X360', 'PS3', 'Wii', 'DS', 'PS'], dtype=object)

In [19]:
#top_6_platform = data[data['platform'].isin(top_6)]

#plot_1 = top_6_platform.plot(kind='bar', grid= True, figsize = (12,5))
#plot_1